In [ ]:
pip install flask-ngrok

In [ ]:
pip install geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 14.8MB 308kB/s 
     |████████████████████████████████| 6.5MB 43.5MB/s 


In [ ]:
pip install geoalchemy2

In [ ]:
pip install shapely_geojson

In [ ]:
pip install flask_cors

In [ ]:
# String de configuração das extensões'
ALLOWED_EXTENSIONS = set(['zip'])

# Extensões permitidas.
def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [ ]:
# pip freeze > requirements.txt

# pip install -r requirements.txt

In [ ]:
import io
import requests
import zipfile
import json
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import psycopg2
import shutil

from flask_cors import CORS, cross_origin
from osgeo import ogr
from shapely_geojson import dumps, Feature
from werkzeug.utils import secure_filename
from flask import Flask, request, render_template, redirect, url_for, render_template, json, Response, jsonify, send_file, make_response
from flask_ngrok import run_with_ngrok
from sqlalchemy import create_engine

app = Flask(__name__)
CORS(app, support_credentials=True)
run_with_ngrok(app)

__file__ = ''
TEMP_FOLDER = os.path.join(os.path.dirname(__file__), "temp")

if os.path.exists(TEMP_FOLDER):
    shutil.rmtree(os.path.join(TEMP_FOLDER))

os.mkdir(TEMP_FOLDER)

app.config['TEMP_FOLDER'] = TEMP_FOLDER

host = ''
port = ''
database = ''
username = ''
password = ''
db_connection_url = ''
filename = ''
path = ''

# 1. HOME -------------------------------------------------------------------
@app.route("/")
def home():
    resp = jsonify(
        {'success': 'true', 'message': 'Conexão com a API realizada com sucesso.'})
    resp.status_code = 201
    return resp


# 2. CONNECTION -------------------------------------------------------------
@app.route("/connection", methods=["POST"])
@cross_origin(supports_credentials=True)
def connection():
    try:
        global host
        global port
        global database
        global username
        global password
        global db_connection_url

        data = request.get_json(force=True, silent=True)

        host = str(data["host"])
        port = str(data["port"])
        database = str(data["database"])
        username = str(data["username"])
        password = str(data["password"])

        con = psycopg2.connect(host=host, database=database,
                               user=username, password=password)
        cur = con.cursor()

        cur.execute('SELECT * FROM pg_catalog.pg_tables')
        recset = cur.fetchall()

        con.close()

        db_connection_url = 'postgres://'+username + \
            ':'+password+'@'+host+':'+port+'/'+database

        resp = jsonify(
            {'success': 'true', 'message': 'Conexão realizada com sucesso.'})
        resp.status_code = 201
        return resp
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro ao realizar conexão.'})
        resp.status_code = 400
        return resp


# 3. TABLES -------------------------------------------------------------
@app.route("/connection/tables", methods=["GET"])
def tables():
    try:
        con = psycopg2.connect(host=host, database=database,
                               user=username, password=password)
        cur = con.cursor()

        cur.execute(
            "Select Tablename From pg_catalog.pg_tables where Schemaname = 'public' and Tableowner <> 'postgres'")
        result = cur.fetchall()

        con.close()

        data = []
        for row in result:
            data.append(row[0])

        resp = jsonify(
            {'success': 'true', 'message': 'Tabelas encontradas com sucesso.', 'tables': data})
        resp.status_code = 201
        return resp
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro ao buscar tabelas.'})
        resp.status_code = 400
        return resp


# 4. COLUMNS -------------------------------------------------------------
@app.route("/connection/columns/<string:table>", methods=["GET"])
def columns(table):
    try:
        con = psycopg2.connect(host=host, database=database,
                               user=username, password=password)
        cur = con.cursor()

        cur.execute(
            f"select column_name from information_schema.columns Where table_name = '{table}' and column_name <> 'id'")
        result = cur.fetchall()

        con.close()

        data = []
        for row in result:
            data.append(row[0])

        resp = jsonify(
            {'success': 'true', 'message': 'Tabelas encontradas com sucesso.', 'tables': data})
        resp.status_code = 201
        return resp
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro ao buscar colunas.'})
        resp.status_code = 400
        return resp

# 5. SHP READ ---------------------------------------------------------------
@app.route("/file", methods=["POST"])
def shpRead():
    try:
        global filename
        global path

        if 'file' not in request.files:
            resp = jsonify(
                {'success': 'false', 'message': 'Nenhum arquivo encontrado na requisição.'})
            resp.status_code = 400
            return resp

        file = request.files['file']

        if file.filename == '':
            resp = jsonify(
                {'success': 'false', 'message': 'Nenhum arquivo selecionado para upload'})
            resp.status_code = 400
            return resp

        if file and allowed_file(file.filename):
            shutil.rmtree(os.path.join(TEMP_FOLDER))
            os.mkdir(TEMP_FOLDER)

            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['TEMP_FOLDER'], filename))

            local_zip = os.path.join(TEMP_FOLDER, filename)
            zip_ref = zipfile.ZipFile(local_zip, 'r')

            format = '.' + filename.rsplit('.', 1)[1].lower()
            name = filename.replace(format, '')

            path = os.path.join(TEMP_FOLDER, name)
            os.mkdir(path)

            zip_ref.extractall(path)
            zip_ref.close()

            shp = ogr.Open(path)
            lyr = shp.GetLayer()

            field_names = [field.name for field in lyr.schema]

            resp = jsonify({
                'message': 'Arquivo lido com sucesso.',
                'file': filename,
                'fields': field_names,
                'field quantity': len(field_names)
            })
            resp.status_code = 201
            return resp
        else:
            resp = jsonify(
                {'success': 'false', 'message': 'Extensão de arquivo não permitida.'})
            resp.status_code = 400
            return resp
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Ocorreu algum problema ao ler o arquivo.'})
        resp.status_code = 400
        return resp


# 6. SHP GENERATE -------------------------------------------------------------
@app.route("/file/<string:table>/<int:id>", methods=["GET"])
def shpGenerate(table, id):
    try:
        shutil.rmtree(os.path.join(TEMP_FOLDER))
        os.mkdir(TEMP_FOLDER)

        engine = create_engine(db_connection_url)

        query = 'Select * From ' + str(table) + ' where Id = ' + str(id)
        shape_postgis = gpd.read_postgis(
            sql=query, con=engine, geom_col="geometry")

        path = os.path.join(TEMP_FOLDER, 'shape')
        os.mkdir(path)

        path_file = os.path.join(TEMP_FOLDER, 'shape', 'shape.shp')
        shape_postgis.to_file(path_file)

        shutil.make_archive(path, 'zip', path)

        FILEPATH = os.path.join(TEMP_FOLDER, 'shape.zip')
        fileobj = io.BytesIO()
        with zipfile.ZipFile(fileobj, 'w') as zip_file:
            zip_info = zipfile.ZipInfo(FILEPATH)
            zip_info.compress_type = zipfile.ZIP_DEFLATED
            with open(FILEPATH, 'rb') as fd:
                zip_file.writestr(zip_info, fd.read())
        fileobj.seek(0)

        response = make_response(fileobj.read())
        response.headers.set('Content-Type', 'zip')
        response.headers.set('Content-Disposition', 'attachment',
                             filename='%s' % os.path.basename(FILEPATH))

        shutil.rmtree(os.path.join(TEMP_FOLDER))
        os.mkdir(TEMP_FOLDER)

        return response
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro não especificado.'})
        resp.status_code = 400
        return resp

# 7. SHP IMAGE -------------------------------------------------------------
@app.route("/image/<string:table>/<int:id>", methods=["GET"])
def shpImage(table, id):
    try:
        shutil.rmtree(os.path.join(TEMP_FOLDER))
        os.mkdir(TEMP_FOLDER)

        engine = create_engine(db_connection_url)

        query = 'Select * From ' + str(table) + ' where Id = ' + str(id)
        shape = gpd.read_postgis(sql=query, con=engine, geom_col="geometry")

        filename = str(table) + '_' + str(id) + '.png'
        path = os.path.join(TEMP_FOLDER, filename)

        shape.plot(figsize=(25, 25))
        plt.savefig(path, dpi=300)

        figfile = io.BytesIO()
        plt.savefig(figfile, format='png')
        figfile.seek(0)

        response = make_response(figfile.getvalue())
        response.headers.set('Content-Type', 'png')
        response.headers.set('Content-Disposition',
                             'attachment', filename=filename)

        shutil.rmtree(os.path.join(TEMP_FOLDER))
        os.mkdir(TEMP_FOLDER)

        return response
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro não especificado.'})
        resp.status_code = 400
        return resp


# 8. STORE ---------------------------------------------------------------
@app.route("/shape/<string:table>", methods=["POST"])
def shpStore(table):
    try:
        shape = gpd.read_file(path)

        data = request.get_json(force=True, silent=True)

        config = data["config"]

        shape = shape.rename(columns=config)

        engine = create_engine(db_connection_url)
        primaryKey = 1

        try:
            con = psycopg2.connect(
                host=host, database=database, user=username, password=password)
            cur = con.cursor()
            cur.execute('Select MAX(id) from ' + str(table))
            id = cur.fetchall()

            primaryKey = id[0][0] + 1
            shape['id'] = primaryKey

            con.close()

            shape.to_postgis(name=table, con=engine, if_exists="append")
        except:
            shape['id'] = primaryKey
            shape.to_postgis(name=table, con=engine, if_exists="replace")

        query = 'Select * From ' + \
            str(table) + ' where Id = ' + str(primaryKey)
        shape = gpd.read_postgis(sql=query, con=engine, geom_col="geometry")

        resp = json.loads(shape.to_json())
        message = json.loads('{ "success": "true", "message" : "Arquivo salvo com sucesso.", "table" : "' +
                             str(table) + '", "id" : "' + str(primaryKey) + '" }')
        resp.update(message)
        return resp, 201
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro ao salvar arquivo.'})
        resp.status_code = 400
        return resp


# 9. INDEX -------------------------------------------------------------
@app.route("/shape/<string:table>", methods=["GET"])
def shpIndex(table):
    try:
        con = psycopg2.connect(host=host, database=database,
                               user=username, password=password)
        cur = con.cursor()
        cur.execute('Select Distinct(Id) From ' + str(table) + ' Order by Id')
        id = cur.fetchall()
        con.close()

        resp = jsonify({'success': 'true', 'message': 'Lista de shapefiles encontrados com sucesso.',
                        'table': table, 'data': id, 'quantity': len(id)})
        resp.status_code = 201
        return resp
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro não especificado.'})
        resp.status_code = 400
        return resp


# 10. SHOW -------------------------------------------------------------
@app.route("/shape/<string:table>/<int:id>", methods=["GET"])
def shpShow(table, id):
    try:
        engine = create_engine(db_connection_url)

        query = 'Select * From ' + \
            str(table) + ' where Id = ' + str(id) + ' limit 1'
        shape = gpd.read_postgis(sql=query, con=engine, geom_col="geometry")

        resp = json.loads(shape.to_json())
        message = json.loads(
            '{ "success": "true", "message" : "Shapefile encontrado com sucesso.", "table" : "' + str(table) + '", "id" : "' + str(id) + '" }')
        resp.update(message)
        return resp, 201
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro não especificado.'})
        resp.status_code = 400
        return resp


# 11. DELETE -------------------------------------------------------------
@app.route("/shape/<string:table>/<int:id>", methods=["DELETE"])
def shpDelete(table, id):
    try:
        con = psycopg2.connect(host=host, database=database,
                               user=username, password=password)
        cur = con.cursor()
        cur.execute('Delete From ' + str(table) + ' where id = ' + str(id))
        con.commit()
        con.close()

        resp = jsonify(
            {'success': 'true', 'message': 'Arquivo apagado com sucesso.', 'table': table, 'id': id})
        resp.status_code = 201
        return resp
    except:
        resp = jsonify(
            {'success': 'false', 'message': 'Erro não especificado.'})
        resp.status_code = 400
        return resp

app.run()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cf67715f9683.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Nov/2020 12:42:17] "POST /connection HTTP/1.1" 201 -
127.0.0.1 - - [06/Nov/2020 12:42:27] "POST /file HTTP/1.1" 201 -
127.0.0.1 - - [06/Nov/2020 12:48:24] "POST /connection HTTP/1.1" 201 -
127.0.0.1 - - [06/Nov/2020 12:48:27] "POST /file HTTP/1.1" 201 -
127.0.0.1 - - [06/Nov/2020 13:00:32] "POST /connection HTTP/1.1" 400 -
